In [431]:
import sys
import os 
import pandas as pd 
import numpy as np
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MultiLabelBinarizer


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from scripts.text_cleaner import clean_text

In [432]:
movie_data = pd.read_csv("../data/movie_metadata.csv")

In [433]:
movie_data.head(20)

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0
5,Color,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,73058679.0,Action|Adventure|Sci-Fi,...,738.0,English,USA,PG-13,263700000.0,2012.0,632.0,6.6,2.35,24000
6,Color,Sam Raimi,392.0,156.0,0.0,4000.0,James Franco,24000.0,336530303.0,Action|Adventure|Romance,...,1902.0,English,USA,PG-13,258000000.0,2007.0,11000.0,6.2,2.35,0
7,Color,Nathan Greno,324.0,100.0,15.0,284.0,Donna Murphy,799.0,200807262.0,Adventure|Animation|Comedy|Family|Fantasy|Musi...,...,387.0,English,USA,PG,260000000.0,2010.0,553.0,7.8,1.85,29000
8,Color,Joss Whedon,635.0,141.0,0.0,19000.0,Robert Downey Jr.,26000.0,458991599.0,Action|Adventure|Sci-Fi,...,1117.0,English,USA,PG-13,250000000.0,2015.0,21000.0,7.5,2.35,118000
9,Color,David Yates,375.0,153.0,282.0,10000.0,Daniel Radcliffe,25000.0,301956980.0,Adventure|Family|Fantasy|Mystery,...,973.0,English,UK,PG,250000000.0,2009.0,11000.0,7.5,2.35,10000


In [434]:
movie_data.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')

In [435]:
movie_data.describe()

,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
count,4993.000000,5028.000000,4939.000000,5020.000000,5036.000000,4.159000e+03,5.043000e+03,5043.000000,5030.000000,5022.000000,4.551000e+03,4935.000000,5030.000000,5043.000000,4714.000000,5043.000000
mean,140.194272,107.201074,686.509212,645.009761,6560.047061,4.846841e+07,8.366816e+04,9699.063851,1.371173,272.770808,3.975262e+07,2002.470517,1651.754473,6.442138,2.220403,7525.964505
std,121.601675,25.197441,2813.328607,1665.041728,15020.759120,6.845299e+07,1.384853e+05,18163.799124,2.013576,377.982886,2.061149e+08,12.474599,4042.438863,1.125116,1.385113,19320.445110
min,1.000000,7.000000,0.000000,0.000000,0.000000,1.620000e+02,5.000000e+00,0.000000,0.000000,1.000000,2.180000e+02,1916.000000,0.000000,1.600000,1.180000,0.000000
25%,50.000000,93.000000,7.000000,133.000000,614.000000,5.340988e+06,8.593500e+03,1411.000000,0.000000,65.000000,6.000000e+06,1999.000000,281.000000,5.800000,1.850000,0.000000
50%,110.000000,103.000000,49.000000,371.500000,988.000000,2.551750e+07,3.435900e+04,3090.000000,1.000000,156.000000,2.000000e+07,2005.000000,595.000000,6.600000,2.350000,166.000000
75%,195.000000,118.000000,194.500000,636.000000,11000.000000,6.230944e+07,9.630900e+04,13756.500000,2.000000,326.000000,4.500000e+07,2011.000000,918.000000,7.200000,2.350000,3000.000000
max,813.000000,511.000000,23000.000000,23000.000000,640000.000000,7.605058e+08,1.689764e+06,656730.000000,43.000000,5060.000000,1.221550e+10,2016.000000,137000.000000,9.500000,16.000000,349000.000000


In [436]:
data = movie_data[['movie_title', 'genres', 'plot_keywords', 'num_critic_for_reviews', 'imdb_score','duration', 'num_user_for_reviews']]

In [437]:
data.head()

,movie_title,genres,plot_keywords,num_critic_for_reviews,imdb_score,duration,num_user_for_reviews
0,Avatar,Action|Adventure|Fantasy|Sci-Fi,avatar|future|marine|native|paraplegic,723.0,7.9,178.0,3054.0
1,Pirates of the Caribbean: At World's End,Action|Adventure|Fantasy,goddess|marriage ceremony|marriage proposal|pi...,302.0,7.1,169.0,1238.0
2,Spectre,Action|Adventure|Thriller,bomb|espionage|sequel|spy|terrorist,602.0,6.8,148.0,994.0
3,The Dark Knight Rises,Action|Thriller,deception|imprisonment|lawlessness|police offi...,813.0,8.5,164.0,2701.0
4,Star Wars: Episode VII - The Force Awakens ...,Documentary,NaN,NaN,7.1,NaN,NaN


In [438]:
data.isna().sum()

movie_title                 0
genres                      0
plot_keywords             153
num_critic_for_reviews     50
imdb_score                  0
duration                   15
num_user_for_reviews       21
dtype: int64

In [439]:
data = data.dropna(axis=0, subset=["plot_keywords"])

In [440]:
print(data.num_critic_for_reviews.median())
print(data.num_critic_for_reviews.mean())
print(data.num_critic_for_reviews.mode())
print(data.num_critic_for_reviews.max())
print(data.num_critic_for_reviews.min())

113.0
143.56508328192473
0    81.0
Name: num_critic_for_reviews, dtype: float64
813.0
1.0


In [441]:
data.num_critic_for_reviews.fillna(data.num_critic_for_reviews.median(), inplace=True)

C:\Users\pc\AppData\Local\Temp\ipykernel_14616\2462707496.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data.num_critic_for_reviews.fillna(data.num_critic_for_reviews.median(), inplace=True)


In [442]:
print(data.num_user_for_reviews.median())
print(data.num_user_for_reviews.max())
print(data.num_user_for_reviews.min())
print(data.num_user_for_reviews.mean())

162.0
5060.0
1.0
280.2215850911325


In [443]:
data.isna().sum()

movie_title               0
genres                    0
plot_keywords             0
num_critic_for_reviews    0
imdb_score                0
duration                  8
num_user_for_reviews      7
dtype: int64

In [444]:
data.num_user_for_reviews.fillna(data.num_user_for_reviews.median(), inplace=True)

C:\Users\pc\AppData\Local\Temp\ipykernel_14616\2776267504.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data.num_user_for_reviews.fillna(data.num_user_for_reviews.median(), inplace=True)


In [445]:
data.isna().sum()

movie_title               0
genres                    0
plot_keywords             0
num_critic_for_reviews    0
imdb_score                0
duration                  8
num_user_for_reviews      0
dtype: int64

In [446]:
data["clean_plot_keywords"] = data["plot_keywords"].apply(clean_text)
data['clean_genres'] = data['genres'].apply(clean_text)

In [447]:
data.head()

,movie_title,genres,plot_keywords,num_critic_for_reviews,imdb_score,duration,num_user_for_reviews,clean_plot_keywords,clean_genres
0,Avatar,Action|Adventure|Fantasy|Sci-Fi,avatar|future|marine|native|paraplegic,723.0,7.9,178.0,3054.0,avatar future marine native paraplegic,Action Adventure Fantasy Sci-Fi
1,Pirates of the Caribbean: At World's End,Action|Adventure|Fantasy,goddess|marriage ceremony|marriage proposal|pi...,302.0,7.1,169.0,1238.0,goddess marriage ceremony marriage proposal pi...,Action Adventure Fantasy
2,Spectre,Action|Adventure|Thriller,bomb|espionage|sequel|spy|terrorist,602.0,6.8,148.0,994.0,bomb espionage sequel spy terrorist,Action Adventure Thriller
3,The Dark Knight Rises,Action|Thriller,deception|imprisonment|lawlessness|police offi...,813.0,8.5,164.0,2701.0,deception imprisonment lawlessness police offi...,Action Thriller
5,John Carter,Action|Adventure|Sci-Fi,alien|american civil war|male nipple|mars|prin...,462.0,6.6,132.0,738.0,alien american civil war male nipple mars prin...,Action Adventure Sci-Fi


In [448]:
data['merged_text_columns'] = ( 
    data['clean_plot_keywords'] + ' ' + data['clean_genres']
)


In [449]:
data = data.drop(columns=['clean_genres', 'clean_plot_keywords'])

In [450]:
scaler = MinMaxScaler()

In [451]:
data['imdb_score'].fillna(data['imdb_score'].mean(), inplace=True)
data['duration'].fillna(data['duration'].mean(), inplace=True)

C:\Users\pc\AppData\Local\Temp\ipykernel_14616\2083979292.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['imdb_score'].fillna(data['imdb_score'].mean(), inplace=True)
C:\Users\pc\AppData\Local\Temp\ipykernel_14616\2083979292.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a co

In [452]:
data[['imdb_score', 'duration']].isna().sum()

imdb_score    0
duration      0
dtype: int64

In [453]:
data[['imdb_score', 'duration']].head

<bound method NDFrame.head of       imdb_score  duration
0            7.9     178.0
1            7.1     169.0
2            6.8     148.0
3            8.5     164.0
5            6.6     132.0
...          ...       ...
5036         7.8      84.0
5037         6.4      95.0
5038         7.7      87.0
5039         7.5      43.0
5042         6.6      90.0

[4890 rows x 2 columns]>

In [454]:
data[['imdb_score_norm', 'duration_norm']] = scaler.fit_transform(data[['imdb_score', 'duration']])

In [455]:
mlb = MultiLabelBinarizer()

In [456]:
data['genres'] = data['genres'].fillna('').astype(str).apply (lambda x: x.split('|'))

genre_encoded = mlb.fit_transform(data['genres'])

genre_df = pd.DataFrame(genre_encoded, columns=mlb.classes_, index=data.index)

data= pd.concat([data, genre_df], axis=1)

data.head

<bound method NDFrame.head of                                     movie_title  \
0                                       Avatar    
1     Pirates of the Caribbean: At World's End    
2                                      Spectre    
3                        The Dark Knight Rises    
5                                  John Carter    
...                                         ...   
5036                           The Mongol King    
5037                                 Newlyweds    
5038                   Signed Sealed Delivered    
5039                 The Following                
5042                         My Date with Drew    

                                    genres  \
0     [Action, Adventure, Fantasy, Sci-Fi]   
1             [Action, Adventure, Fantasy]   
2            [Action, Adventure, Thriller]   
3                       [Action, Thriller]   
5              [Action, Adventure, Sci-Fi]   
...                                    ...   
5036                        [Crime,

In [457]:
print(mlb.classes_)

['Action' 'Adventure' 'Animation' 'Biography' 'Comedy' 'Crime'
 'Documentary' 'Drama' 'Family' 'Fantasy' 'Film-Noir' 'Game-Show'
 'History' 'Horror' 'Music' 'Musical' 'Mystery' 'News' 'Reality-TV'
 'Romance' 'Sci-Fi' 'Short' 'Sport' 'Thriller' 'War' 'Western']


In [458]:
data.columns

Index(['movie_title', 'genres', 'plot_keywords', 'num_critic_for_reviews',
       'imdb_score', 'duration', 'num_user_for_reviews', 'merged_text_columns',
       'imdb_score_norm', 'duration_norm', 'Action', 'Adventure', 'Animation',
       'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family',
       'Fantasy', 'Film-Noir', 'Game-Show', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',
       'Short', 'Sport', 'Thriller', 'War', 'Western'],
      dtype='object')

In [459]:
numeric_data = data[['imdb_score_norm', 'duration_norm']]

In [460]:
numeric_data.head()

,imdb_score_norm,duration_norm
0,0.818182,0.339286
1,0.714286,0.321429
2,0.675325,0.279762
3,0.896104,0.311508
5,0.649351,0.248016


In [461]:
data.head()

,movie_title,genres,plot_keywords,num_critic_for_reviews,imdb_score,duration,num_user_for_reviews,merged_text_columns,imdb_score_norm,duration_norm,...,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Thriller,War,Western
0,Avatar,"[Action, Adventure, Fantasy, Sci-Fi]",avatar|future|marine|native|paraplegic,723.0,7.9,178.0,3054.0,avatar future marine native paraplegic Action ...,0.818182,0.339286,...,0,0,0,0,1,0,0,0,0,0
1,Pirates of the Caribbean: At World's End,"[Action, Adventure, Fantasy]",goddess|marriage ceremony|marriage proposal|pi...,302.0,7.1,169.0,1238.0,goddess marriage ceremony marriage proposal pi...,0.714286,0.321429,...,0,0,0,0,0,0,0,0,0,0
2,Spectre,"[Action, Adventure, Thriller]",bomb|espionage|sequel|spy|terrorist,602.0,6.8,148.0,994.0,bomb espionage sequel spy terrorist Action Adv...,0.675325,0.279762,...,0,0,0,0,0,0,0,1,0,0
3,The Dark Knight Rises,"[Action, Thriller]",deception|imprisonment|lawlessness|police offi...,813.0,8.5,164.0,2701.0,deception imprisonment lawlessness police offi...,0.896104,0.311508,...,0,0,0,0,0,0,0,1,0,0
5,John Carter,"[Action, Adventure, Sci-Fi]",alien|american civil war|male nipple|mars|prin...,462.0,6.6,132.0,738.0,alien american civil war male nipple mars prin...,0.649351,0.248016,...,0,0,0,0,1,0,0,0,0,0


In [462]:
data = data.drop(columns=['plot_keywords', 'num_critic_for_reviews', 'num_user_for_reviews',])

In [463]:
data.columns

Index(['movie_title', 'genres', 'imdb_score', 'duration',
       'merged_text_columns', 'imdb_score_norm', 'duration_norm', 'Action',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Film-Noir', 'Game-Show', 'History',
       'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV',
       'Romance', 'Sci-Fi', 'Short', 'Sport', 'Thriller', 'War', 'Western'],
      dtype='object')

In [464]:
print(data['movie_title'].head(10))

0                                       Avatar 
1     Pirates of the Caribbean: At World's End 
2                                      Spectre 
3                        The Dark Knight Rises 
5                                  John Carter 
6                                 Spider-Man 3 
7                                      Tangled 
8                      Avengers: Age of Ultron 
9       Harry Potter and the Half-Blood Prince 
10          Batman v Superman: Dawn of Justice 
Name: movie_title, dtype: object


In [465]:
data.to_csv("../data/cleaned_data.csv", index=False)
genre_df.to_csv("../data/genre_df.csv", index=False)
numeric_data.to_csv("../data/numeric_data.csv", index=False)